### Milestone 3: Traditional statistical and machine learning methods, due Wednesday, April 19, 2017

Think about how you would address the genre prediction problem with traditional statistical or machine learning methods. This includes everything you learned about modeling in this course before the deep learning part. Implement your ideas and compare different classifiers. Report your results and discuss what challenges you faced and how you overcame them. What works and what does not? If there are parts that do not work as expected, make sure to discuss briefly what you think is the cause and how you would address this if you would have more time and resources. 

You do not necessarily need to use the movie posters for this step, but even without a background in computer vision, there are very simple features you can extract from the posters to help guide a traditional machine learning model. Think about the PCA lecture for example, or how to use clustering to extract color information. In addition to considering the movie posters it would be worthwhile to have a look at the metadata that IMDb provides. 

You could use Spark and the [ML library](https://spark.apache.org/docs/latest/ml-features.html#word2vec) to build your model features from the data. This may be especially beneficial if you use additional data, e.g., in text form.

You also need to think about how you are going to evaluate your classifier. Which metrics or scores will you report to show how good the performance is?

The notebook to submit this week should at least include:

- Detailed description and implementation of two different models
- Description of your performance metrics
- Careful performance evaluations for both models
- Visualizations of the metrics for performance evaluation
- Discussion of the differences between the models, their strengths, weaknesses, etc. 
- Discussion of the performances you achieved, and how you might be able to improve them in the future

#### Preliminary Peer Assessment

It is important to provide positive feedback to people who truly worked hard for the good of the team and to also make suggestions to those you perceived not to be working as effectively on team tasks. We ask you to provide an honest assessment of the contributions of the members of your team, including yourself. The feedback you provide should reflect your judgment of each team member’s:

- Preparation – were they prepared during team meetings?
- Contribution – did they contribute productively to the team discussion and work?
- Respect for others’ ideas – did they encourage others to contribute their ideas?
- Flexibility – were they flexible when disagreements occurred?

Your teammate’s assessment of your contributions and the accuracy of your self-assessment will be considered as part of your overall project score.

Preliminary Peer Assessment: [https://goo.gl/forms/WOYC7pwRCSU0yV3l1](https://goo.gl/forms/WOYC7pwRCSU0yV3l1)

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn import tree
from sklearn import ensemble
from sklearn.cross_validation import KFold
import StringIO
from PIL import Image
import urllib
from random import sample
from imdb import IMDb
import tmdbsimple as tmdb
import time
tmdb.API_KEY = 'c5d41f08e55fca6e9f5fc0b6d1735540'

In [21]:
#read in genre and movie id multi class
mmc = pd.read_csv('C:\Users\Chad\Desktop\Data Science 2\Project\ms3\y_labels_multiclass2.csv')

#access pull infomration from api with smaller subset first 
#rowindex =  np.array(sample(xrange(len(mmc)), 2000))
#mmcr = mmc.ix[rowindex]
#mmcr = mmcr.reset_index(drop=True)


In [22]:
mmcr.head()

,movie_id,genre_id,genre_name
0,57119,35,Comedy
1,18607,35,Comedy
2,157845,53,Thriller
3,37786,10749,Romance
4,164558,99,Documentary


In [10]:
def getMovies(tmdb_id):
    movie = tmdb.Movies(tmdb_id)
    return movie.info()

In [20]:
#found a nice example of extracting dominate color palette from image, modify a bit for our project
#http://stackoverflow.com/a/16216866/190597 (Jaime)
#http://stackoverflow.com/a/16840350/190597 (Jaime)

def palette(img):
    arr = np.asarray(img)
    palette, index = np.unique(asvoid(arr).ravel(), return_inverse=True)
    palette = palette.view(arr.dtype).reshape(-1, arr.shape[-1])
    count = np.bincount(index)
    order = np.argsort(count)
    return palette[order[::-1]]

def asvoid(arr):
    arr = np.ascontiguousarray(arr)
    return arr.view(np.dtype((np.void, arr.dtype.itemsize * arr.shape[-1])))


In [ ]:
#generate a pallatte score for the top 5 dominate color in this image
#calculate color intensity for top 5 dominate color
#calcuate average red, blue, green scale
def getPaletteScore(mpath):
    base_url = "https://image.tmdb.org/t/p/"
    file_size = "w500"
    poster_path = mpath 
    poster_url = base_url+file_size+poster_path
    img = Image.open(urllib.urlopen(poster_url), 'r').convert('RGB')
    palettescore = np.sum(palette(img)[:5], axis = 1)
    avgRBG = np.mean(palette(img)[:5], axis = 0)
    return palettescore, avgRBG

In [82]:
idx = range(0, len(mmc))
cols = ['genre_name', 'id', 'popularity', 'budget', 'revenue', 'pal_intensity_1', 'pal_intensity_2', 'pal_intensity_3', 
        'pal_intensity_4', 'pal_intensity_5', 'avg_red', 'avg_blue', 'avg_green']
df = pd.DataFrame(index=idx, columns=cols)

for i in range(0,len(mmc)):
    movie = getMovies(mmc.ix[i, 'movie_id'])
    pal_score, avgRBG = getPaletteScore(movie['poster_path'])
    df['genre_name'][i] = mmc.ix[i, 'genre_name']
    df['id'][i] = mmc.ix[i, 'movie_id']
    df['popularity'][i] = movie['popularity']
    df['budget'][i] = movie['budget']
    df['revenue'][i] = movie['revenue']
    df['pal_intensity_1'][i] = pal_score[0]
    df['pal_intensity_2'][i] = pal_score[1]
    df['pal_intensity_3'][i] = pal_score[2]
    df['pal_intensity_4'][i] = pal_score[3]
    df['pal_intensity_5'][i] = pal_score[4]
    df['avg_red'][i] = avgRBG[0]
    df['avg_blue'][i] = avgRBG[1]
    df['avg_green'][i] = avgRBG[2]

In [83]:
#save to csv
df.to_csv('C:\Users\Chad\Desktop\Data Science 2\Project\ms3\movie_df.csv', sep=',')

In [85]:
def split(data):
    length = data.shape[0]
    indices = range(length)
    np.random.shuffle(indices)
    train_indices = indices[0:int(length * .7)]
    test_indices = indices[int(length * .7):]
    train = data.iloc[train_indices]
    test = data.iloc[test_indices]
    return train, test


In [88]:
train, test = split(df)
y_train = train.iloc[:, 0]
x_train = train.iloc[:, 2:]
y_test = test.iloc[:, 0]
x_test = test.iloc[:, 2:]

In [89]:
n_trees = np.arange(1, 151, 5)  
depths = np.arange(1, 25)   

# To keep track of the best model
best_score = 0

# Run grid search for model with 5-fold cross validation
print '5-fold cross validation:'

for trees in n_trees:
    for depth in depths:
        
        # Cross validation for every experiment
        k_folds = KFold(train.shape[0], n_folds=5, shuffle=True)
        scores = []
        for train_indices, validation_indices in k_folds:
            # Generate training data
            x_train_cv = x_train.iloc[train_indices]
            y_train_cv = y_train.iloc[train_indices]
            # Generate validation data
            x_validate = x_train.iloc[validation_indices]
            y_validate = y_train.iloc[validation_indices]
            
            # Fit random forest on training data
            model = ensemble.RandomForestClassifier(n_estimators=trees, max_depth=depth)
            model.fit(x_train_cv, y_train_cv)
            # Score on validation data
            scores += [model.score(x_validate, y_validate)]
        
        # Record and report accuracy
        average_score = np.mean(scores)
        #print "Trees:", trees, "Depth:", depth, "Score:", average_score
        
        # Update our record of the best parameters see so far
        if average_score > best_score:
            best_score = average_score
            best_trees = trees
            best_depth = depth

# Fit model on entire train set using chosen number of trees and depth
model = ensemble.RandomForestClassifier(n_estimators=best_trees, max_depth=best_depth)
model.fit(x_train, y_train)

print 'Chosen number of trees, depth:', best_trees, ',', best_depth
print 'Test accuracy:', model.score(x_test, y_test)

5-fold cross validation:
Chosen number of trees, depth: 91 , 10
Test accuracy: 0.351351351351


In [92]:
preds = model.predict(x_test)
pd.crosstab(y_test, preds, rownames=['actual'], colnames=['preds'])

preds,Action,Animation,Comedy,Documentary,Drama,Romance,Thriller
actual,,,,,,,
Action,237,7,100,0,2,15,154
Animation,38,26,81,1,0,6,31
Comedy,88,10,361,4,5,24,84
Documentary,10,0,54,2,1,2,25
Drama,39,5,123,1,3,8,67
Romance,70,7,207,1,0,22,70
Thriller,117,2,89,0,1,11,194


In [ ]:
mml = 